In [1]:
import numpy as np
import cv2
import pickle
import os
from PIL import Image
import time
import copy
from core.util import load_pkl, write_pkl
from core.util.ReSample import resize

class toFile:
    def __init__(self, folder, n_images_per_file, suffix):
        self.folder = folder
        self.n_images_per_file = n_images_per_file
        self.ct = 0
        self.cache = []
        self.suffix = suffix
    
    def tofile(self, X):
        a = self.ct
        if len(self.cache) > 0:
            X = np.concatenate([self.cache, X], axis=0)
            self.cache = []
        for i in range(0, X.shape[0], self.n_images_per_file):
            if len(X[i:i+self.n_images_per_file]) != self.n_images_per_file:
                self.cache = X[i:i+self.n_images_per_file]
                break
            write_pkl(self.folder+'/'+str(self.ct)+'.'+self.suffix, X[i:i+self.n_images_per_file])
            self.ct += 1
#         print('<INFO> write %d files, current total files %d, %d images in buffer'%(self.ct-a, self.ct, len(self.cache)))
        return True


In [2]:
class PreProcess():
    def __init__(self, output_folder, n_images_per_file=1000, suffix='spatial_raw', size_list=[8,16,32,64,128,256], stride=123, downsample_level=3):
        self.g_count = 0
        self.output_folder = output_folder
        self.size_list = size_list
        self.n_images_per_file = n_images_per_file
        # output folder has a series subfolder 8/16/32/64/128/256
        os.system('mkdir '+self.output_folder)
        self.tofile = []
        for i in size_list:
            os.system('mkdir '+self.output_folder+'/'+str(i))
            self.tofile.append(toFile(self.output_folder+'/'+str(i)+'/', n_images_per_file, suffix))
        self.stride = stride
        self.downsample_level = downsample_level
    
    def patch_to_file(self, patch):
        for i in range(len(self.size_list)):
            patch = resize(patch, self.size_list[i])
            flag = self.tofile[i].tofile(patch)
            if flag == False:
                return 
#             flag = cv2.imwrite(self.output_folder+'/'+str(i)+'/'+str(self.g_count)+'.png', patch[0])
        self.g_count += 1
        
    def crop_image(self, x):
        for i in range(0, x.shape[0], self.stride):
            for j in range(0, x.shape[1], self.stride):
                patch = x[i:i+256, j:j+256]
                if patch.shape[0] == 256 and patch.shape[1] == 256:
                    self.patch_to_file(patch.reshape(1, patch.shape[0], patch.shape[1], -1))

    
    def process_folder(self, folder):
        init_count, t0 = self.g_count, time.time()
        files = os.listdir(folder)
        for file in files:
            try:
                x = cv2.imread(folder+'/'+file)
                x.shape
            except:
                continue
            x = x.reshape(1, x.shape[0], x.shape[1], -1)
            for level in range(self.downsample_level+1):
                self.crop_image(x[0])
                x = resize(x, x.shape[1]//2)
        print('Processed folder=%s, get %d images using %f sec, total images=%d'
                  %(folder, self.g_count-init_count, time.time()-t0, self.g_count))
        

In [3]:
pp = PreProcess(output_folder='/Users/alex/Desktop//unit/data/', n_images_per_file=100, stride=256, downsample_level=0) 

In [4]:
pp.process_folder('/Users/alex/Desktop/proj/data/train256/')

Processed folder=/Users/alex/Desktop/proj/data/train256/, get 1427 images using 16.424568 sec, total images=1427


In [5]:
4*0.75/0.25


12.0